# Step 2: Extract Marking Scheme to Excel
Use Gemini to parse the Word marking scheme into structured Excel sheets. Configure the exam `prefix` and dataset folder in the next cell before running.

In [1]:
from grading_utils import setup_paths

prefix = "VTC Test"
paths = setup_paths(prefix, "sample")

marking_scheme_word_file = f"../sample/{prefix} Marking Scheme.docx"
marking_scheme_excel_file = paths["marking_scheme_file"]

In [2]:
from grading_utils import init_gemini_client
from google.genai import types

# Initialize Gemini client
client = init_gemini_client()

✓ Vertex AI Express Mode initialized


In [3]:
import mammoth
import html2text
from IPython.display import Markdown, display

# Convert .docx to HTML using mammoth
with open(marking_scheme_word_file, "rb") as docx_file:
    result = mammoth.convert_to_html(docx_file)
    html_content = result.value

# Convert HTML to markdown using html2text
h = html2text.HTML2Text()
h.ignore_links = False
h.body_width = 0  # Don't wrap text

markdown_content = h.handle(html_content)

# Display as formatted markdown
display(Markdown(markdown_content))

**Detailed Marking Scheme (0-10 Scale)**

Use the following rubric to determine the score for each q.

**Q1: The Role of VTC**  
The VTC is the largest provider of **VPET** in Hong Kong. Briefly explain what **VPET** stands for and why it is important for Hong Kong’s workforce development.

  * **Answer:** VPET stands for **Vocational and Professional Education and Training**. It is important because it provides students with practical skills and specialized knowledge needed by industries, ensuring Hong Kong has a skilled labor force to support the economy.
  * **Marking Breakdown:**
    * **[2 marks]** Correctly stating "Vocational and Professional Education and Training".
    * **[4 marks]** Explaining that it focuses on _practical skills_ or _specialized trades_.
    * **[4 marks]** Explaining the benefit to the workforce (reducing skills gap, employment readiness).



**Q2: Member Institutions**  
Compare **IVE (Hong Kong Institute of Vocational Education)** and **THEi (Technological and Higher Education Institute of Hong Kong)**. What is the main difference between the types of qualifications/programmes offered by these two institutions?

  * **Answer:** **IVE** primarily focuses on **Higher Diploma (HD)** programmes which are practical and technical in nature. **THEi** focuses on vocationally-oriented **Bachelor’s Degree** programmes that combine practical application with higher-level theory.
  * **Marking Breakdown:**
    * **[5 marks]** Correctly identifying that IVE offers Higher Diplomas/Technical training.
    * **[5 marks]** Correctly identifying that THEi offers Bachelor's Degrees.



**Q3: Educational Philosophy**  
VTC emphasizes the **" Think and Do"** approach. Explain what this phrase means in the context of a student's learning experience.

  * **Answer:** This approach cultivates the ability to apply brainpower (theory/thinking) to practical work (doing). It ensures students understand the theory behind their work while also possessing the hands-on technical skills to execute it.
  * **Marking Breakdown:**
    * **[3 marks]** Explaining "Think" (Theory/Academic knowledge).
    * **[3 marks]** Explaining "Do" (Practical skills/Hands-on).
    * **[4 marks]** Explaining the synthesis: Being able to solve problems by combining both head and hands.



**Q4: Study Pathways**  
If a Secondary 6 student does **not** achieve the minimum entrance requirements for a Bachelor's Degree or a Higher Diploma, what is the VTC study pathway available to them to eventually reach a Higher Diploma level? (Name the specific foundation programme).

  * **Answer:** The student can enroll in the **Diploma of Foundation Studies (DFS)** (or Diploma of Vocational Education). Upon successful completion of this diploma, they are eligible to progress to VTC **Higher Diploma (HD)** programmes.
  * **Marking Breakdown:**
    * **[5 marks]** Correctly naming the "Diploma of Foundation Studies" (DFS) or "Diploma of Vocational Education" (DVE).
    * **[5 marks]** Explanation of the progression (Completion of DFS guarantees/allows entry to Higher Diploma).



**Q5: Industry Partnership**  
Why does the VTC collaborate closely with industry partners (companies and trade associations)? Give **two** examples of how this benefits students.

  * **Answer:** Collaboration ensures the curriculum is up-to-date with market trends. Benefits include: (1) Internship/Work-integrated learning opportunities, (2) Job placement support, (3) Access to industry-standard equipment/facilities.
  * **Marking Breakdown:**
    * **[4 marks]** General explanation (Curriculum relevance/Industry needs).
    * **[3 marks]** First specific benefit (e.g., Internships).
    * **[3 marks]** Second specific benefit (e.g., Job prospects or Equipment).



**General Grading Guide (for partial marks)**

  * **9-10 marks:** The answer is complete, accurate, uses correct terminology, and is well-explained.
  * **6-8 marks:** The answer is mostly correct but misses a specific detail (e.g., forgets the full name of a diploma) or the explanation is slightly vague.
  * **3-5 marks:** The student shows basic understanding but misses the core point or only answers half the q.
  * **0-2 marks:** The answer is largely incorrect, irrelevant, or blank.




In [4]:
from pydantic import BaseModel, Field
from typing import List
import json
import pandas as pd

# Define Pydantic models for structured output
class Question(BaseModel):
    """Represents a single question with answer and marking scheme"""
    question_number: str = Field(description="The question number (e.g., '1', '2', '22a', '22b', 'Q1','Q2')")
    question_text: str = Field(description="The full question text")
    marking_scheme: str = Field(description="Well-formatted marking scheme using markdown. Use bullet points (-), numbered lists (1., 2.), bold (**text**) for key terms, and clear line breaks. Include point allocations in parentheses (e.g., '- Key concept explained (2 marks)'). Structure should be clear and scannable.")
    marks: int = Field(description="Total marks available for this question")

class MarkingSchemeResponse(BaseModel):
    """Wrapper class for list of questions and general grading guide"""
    general_grading_guide: str = Field(default="", description="General grading guide for partial marks applicable to all questions, formatted in markdown")
    questions: List[Question] = Field(description="List of questions with marking schemes and marks")

# Create a prompt for Gemini to extract structured data
prompt = f"""
Please analyze this marking scheme document and extract structured, well-formatted data.

**FORMATTING REQUIREMENTS for marking_scheme:**
- Use markdown formatting (bullet points -, numbered lists 1., 2., bold **text**)
- Each marking criterion should be on its own line
- Show point allocations clearly (e.g., "- Correct formula (2 marks)")
- Use clear hierarchy with proper indentation for sub-points
- Add line breaks between major sections
- Bold important terms or key concepts
- Make it scannable and easy to read

**EXTRACT:**

1. **GENERAL GRADING GUIDE**: Extract any general grading guide or guidance for partial marks that applies to all/multiple questions (use markdown formatting)

2. **FOR EACH QUESTION**: Extract:
   - Question number
   - Question text
   - **Marking scheme** (well-formatted with markdown, bullets, numbering, clear point allocation)
   - Total marks available

**Important**: When extracting the marking_scheme for each question, incorporate any general grading principles that apply to that question's scoring.

**Document:**

{markdown_content}
"""

# Create configuration with structured output
config = types.GenerateContentConfig(
    temperature=0,
    top_p=0.5,
    max_output_tokens=4096,
    response_mime_type="application/json",
    response_schema=MarkingSchemeResponse,
)

# Send to Gemini using structured output
response = client.models.generate_content(
    model="gemini-3-flash-preview",
    contents=[{"role": "user", "parts": [{"text": prompt}]}],
    config=config,
)

# Extract parsed response
if hasattr(response, 'parsed') and response.parsed is not None:
    result = response.parsed
    general_guide = result.general_grading_guide
    questions_data = [q.model_dump() for q in result.questions]
    print(f"✓ Successfully extracted {len(questions_data)} questions with structured output!")
    if general_guide:
        print(f"\n📋 General Grading Guide:\n{general_guide}\n")
else:
    # Fallback to text parsing
    response_text = response.text
    try:
        parsed_json = json.loads(response_text)
        general_guide = parsed_json.get('general_grading_guide', '')
        questions_data = parsed_json.get('questions', [])
        print(f"✓ Successfully extracted {len(questions_data)} questions from text!")
        if general_guide:
            print(f"\n📋 General Grading Guide:\n{general_guide}\n")
    except json.JSONDecodeError as e:
        print("Error extracting questions:")
        print(response_text)
        questions_data = []
        general_guide = ""

# Append general grading guide to each marking_scheme
if general_guide:
    for question in questions_data:
        # Add general guide with markdown formatting
        question['marking_scheme'] = f"{question['marking_scheme']}\n\n---\n\n**General Grading Guide:**\n{general_guide}"

# Convert to DataFrame and display
if questions_data:
    df = pd.DataFrame(questions_data)
    display(df)
else:
    print("No questions extracted")

✓ Successfully extracted 5 questions with structured output!

📋 General Grading Guide:
### General Rubric for Partial Marks (0-10 Scale)

- **9-10 marks**: The answer is complete, accurate, uses correct terminology, and is well-explained.
- **6-8 marks**: The answer is mostly correct but misses a specific detail (e.g., forgets the full name of a diploma) or the explanation is slightly vague.
- **3-5 marks**: The student shows basic understanding but misses the core point or only answers half the question.
- **0-2 marks**: The answer is largely incorrect, irrelevant, or blank.



,question_number,question_text,marking_scheme,marks
0,Q1,The VTC is the largest provider of VPET in Hon...,- **Definition (2 marks)**: Correctly stating ...,10
1,Q2,Compare IVE (Hong Kong Institute of Vocational...,- **IVE Qualification (5 marks)**: Correctly i...,10
2,Q3,"VTC emphasizes the ""Think and Do"" approach. Ex...","- **""Think"" Component (3 marks)**: Explaining ...",10
3,Q4,If a Secondary 6 student does not achieve the ...,- **Programme Identification (5 marks)**: Corr...,10
4,Q5,Why does the VTC collaborate closely with indu...,- **General Rationale (4 marks)**: Explaining ...,10


In [5]:
from termcolor import colored

# Validate that all marking schemes are non-empty
if questions_data:
    empty_schemes = []
    for question in questions_data:
        marking_scheme = question.get('marking_scheme', '').strip()
        if not marking_scheme:
            empty_schemes.append(question.get('question_number', 'Unknown'))
    
    if empty_schemes:
        print("\n" + "="*60)
        print(colored("❌ ERROR: Empty marking schemes detected!", "red", attrs=['bold']))
        print("="*60)
        print(colored(f"\nThe following questions have empty marking schemes:", "red"))
        for q_num in empty_schemes:
            print(colored(f"  • Question {q_num}", "red"))
        print(colored(f"\n⚠️  Please fix the marking scheme document and try again.", "yellow"))
        print(colored("⚠️  All questions must have a marking scheme before proceeding.", "yellow"))
        print("="*60 + "\n")
        
        # Stop execution by raising an error
        raise ValueError(f"Validation failed: {len(empty_schemes)} question(s) have empty marking schemes")
    else:
        print(colored("✓ Validation passed: All marking schemes are non-empty", "green"))
else:
    print(colored("❌ ERROR: No questions extracted from document!", "red", attrs=['bold']))
    raise ValueError("No questions data to validate")

✓ Validation passed: All marking schemes are non-empty


In [6]:
if questions_data:
    # Create Excel writer with multiple sheets
    with pd.ExcelWriter(marking_scheme_excel_file, engine='openpyxl') as writer:
        # Sheet 1: Marking Scheme (detailed rubric with general guide)
        df.to_excel(writer, sheet_name='Marking Scheme', index=False)
        
        # Sheet 2: Summary
        summary_data = {
            'Total Questions': [len(questions_data)],
            'Total Marks': [df['marks'].sum()],
            'Output File': [marking_scheme_excel_file]
        }
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_excel(writer, sheet_name='Summary', index=False)
    
    print(f"✓ Successfully saved marking scheme to: {marking_scheme_excel_file}")
    print(f"  - Questions: {len(questions_data)}")
    print(f"  - Total marks: {df['marks'].sum()}")
    print(f"  - General Grading Guide: Appended to each marking_scheme")
else:
    print("No data to save")

✓ Successfully saved marking scheme to: ../sample/VTC Test Marking Scheme.xlsx
  - Questions: 5
  - Total marks: 50
  - General Grading Guide: Appended to each marking_scheme
